# Data ETL

## Import Modules

In [1]:
import os
import sys
sys.path.insert(0,os.path.join("..","..","Resources","AccessInformation"))

from accessinformation import access_token
from stravalib import Client , unithelper

import pandas as pd
import numpy as np

client = Client(access_token =access_token)

## Structuring Data for JSON Export

In [2]:
print(f'Runner information: {client.get_athlete().firstname} {client.get_athlete().lastname}')

Runner information: Raul Maldonado


### Functions

In [3]:
'''
Function: Athlete Activies

Input: client 
Response: Get activities associated with client from related access_token in client instance above
'''
def AthleteActivities(client):
    return(client.get_activities())


In [4]:
'''
Function: dictionaryOfDf_toDF

Input: dictionary 
Response: convert dictionary of Data Frames to one single datagrame by the pandas' concat function
'''
def dictionaryOfDF_toDF(diction):
    listOfDataFrames = list(diction.values())
    df = pd.concat(listOfDataFrames)
    return(df)

In [5]:
types = ['time', 'heartrate','distance', 'latlng', 'altitude', 'velocity_smooth', 
                                                'moving', 'grade_smooth', 'temp']


'''
Function: getActivityStream

Input: Activities object from instance of GetActivities from client
Response: dictionary of dataframes by required types from list "types", seen above.
'''
def getActivityStream(activitiesObject):
    dataFrameDictionary = {}
    
    #Create Dataframe for each activity in the activities object
    for activity in activitiesObject:
        try:
            '''
            Get activity id, name, and types of activity streams from list "types"
            '''
            actID = activity.id
            actName = activity.name
            streamObject = client.get_activity_streams(actID, types=types, series_type='time')
            actID_list = [actID for i in range(len(streamObject['latlng'].data))]
            '''
            Instantiate pandas dataframe with required type features. Moreover, Latitude, Longitude, and Date's are engineered
            into the dataframe
            '''
            df=pd.DataFrame( {"ActivityID": actID_list, "LatLong": streamObject['latlng'].data, "Time": streamObject['time'].data ,
                              "HeartRate": streamObject['heartrate'].data, "Distance": streamObject['distance'].data,
                              "Altitude": streamObject['altitude'].data,"Grade_Smooth": streamObject['grade_smooth'].data,
                              'Moving': streamObject['moving'].data ,"Velocity_Smooth":streamObject['velocity_smooth'].data})
            
            
            df['Latitude'] = df['LatLong'].apply(lambda x: x[0])
            df['Longitude'] = df['LatLong'].apply(lambda x: x[1])
            df.drop('LatLong',inplace = True,axis=1)
            
            df["Date"] = activity.start_date.date()
        
        except KeyError as e:
            continue
        
        dataFrameDictionary[f'{actName.replace(" ","")}-{actID}'] = df
    return(dataFrameDictionary)

In [6]:
dataframe_dictionary = getActivityStream(AthleteActivities(client))

In [18]:
unithelper.miles(dataframe_dictionary['MorningRun-1654752525'].Distance)

Quantity(0         0.0
1        15.2
2        35.5
3        40.4
4        60.6
5        83.0
6        91.3
7       113.2
8       135.8
9       156.0
10      179.3
11      188.0
12      209.9
13      219.0
14      241.8
15      262.9
16      283.5
17      298.4
18      318.9
19      330.7
20      352.9
21      374.9
22      380.7
23      397.2
24      403.8
25      410.0
26      417.5
27      423.0
28      435.2
29      458.0
        ...  
271    4510.5
272    4532.9
273    4541.2
274    4562.9
275    4578.6
276    4599.6
277    4602.0
278    4620.6
279    4642.3
280    4656.4
281    4671.0
282    4691.3
283    4713.3
284    4733.9
285    4756.3
286    4777.3
287    4786.7
288    4801.8
289    4803.9
290    4821.7
291    4829.6
292    4850.0
293    4871.5
294    4893.5
295    4895.2
296    4914.1
297    4933.8
298    4943.9
299    4963.9
300    4975.0
Name: Distance, Length: 301, dtype: float64, NamedComposedUnit('mi', ComposedUnit([NamedComposedUnit('fur', ComposedUnit([NamedComposedUn

# Export Dictionary of Dataframes to Json files

In [7]:
for key,values in dataframe_dictionary.items():
    values.to_json(f"../../Data/JSONData/{key}.json", orient='records')

In [8]:
main_df = dictionaryOfDF_toDF(dataframe_dictionary)
main_df.to_json(f"../../Data/JSONData/MainDataset.json", orient='records')

In [13]:
main_df.head()

,ActivityID,Time,HeartRate,Distance,Altitude,Grade_Smooth,Moving,Velocity_Smooth,Latitude,Longitude,Date
0,1654752525,0,118,0.0,63.2,-6.8,False,0.0,37.721417,-122.472470,2018-06-22
1,1654752525,7,121,15.2,62.2,-6.7,True,2.2,37.721397,-122.472641,2018-06-22
2,1654752525,14,119,35.5,60.8,-6.4,True,2.5,37.721386,-122.472872,2018-06-22
3,1654752525,16,123,40.4,60.5,-6.0,True,2.8,37.721386,-122.472928,2018-06-22
4,1654752525,24,123,60.6,59.3,-5.7,True,2.5,37.721321,-122.473142,2018-06-22


## Data to SQL database

In [10]:
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, ForeignKey
from sqlalchemy import Column, Date, Integer, String
from sqlalchemy.ext.declarative import declarative_base

import pandas as pd

engine = create_engine('sqlite:///../../Analysis/db/RunningData.db')

In [11]:
'''
Import Running data into the  "RunningData" SQLite database table Segments.
'''

Base = declarative_base()

class Segments(Base):
    __tablename__ = "Segments"
    ids = Column(Integer, primary_key = True)
    activityID = Column(Integer)
    time = Column(Integer)
    distance = Column(Integer)
    heartrate = Column(Integer)
    altitude = Column(Integer)
    grade_smooth = Column(Integer)
    moving = Column(Integer)
    velocity_smooth = Column(Integer)
    latitude = Column(Integer)
    longitude = Column(Integer)
    
    def __init__(self, activityID, time, heartrate, distance, altitude, grade_smooth, moving, \
                velocity_smooth, latitude, longitude):
        self.activityID = activityID
        self.time = time
        self.heartrate = heartrate
        self.distance = distance
        self.altitude = altitude 
        self.grade_smooth = grade_smooth
        self.moving = moving
        self.velocity_smooth = velocity_smooth
        self.latitude = latitude
        self.longitude = longitude
        
Base.metadata.create_all(engine)

session = Session(bind = engine)
for index, rows in main_df.iterrows():
    session.add(Segments( activityID =rows['ActivityID'],time = rows['Time'], heartrate = rows['HeartRate'],distance = rows['Distance'], 
                         altitude = rows['Altitude'], grade_smooth = rows['Grade_Smooth'], moving = rows['Moving'],
                         velocity_smooth = rows['Velocity_Smooth'],latitude = rows['Latitude'],longitude = rows['Longitude']))

session.commit()

In [12]:
#Test values are in database
[i for i in engine.execute('SELECT * FROM Segments LIMIT 10;')]

[(1, 1654752525, 0, 0, 118, 63.2, -6.8, 0, 0, 37.721417, -122.47247),
 (2, 1654752525, 7, 15.2, 121, 62.2, -6.7, 1, 2.2, 37.721397, -122.472641),
 (3, 1654752525, 14, 35.5, 119, 60.8, -6.4, 1, 2.5, 37.721386, -122.472872),
 (4, 1654752525, 16, 40.4, 123, 60.5, -6, 1, 2.8, 37.721386, -122.472928),
 (5, 1654752525, 24, 60.6, 123, 59.3, -5.7, 1, 2.5, 37.721321, -122.473142),
 (6, 1654752525, 33, 83, 124, 58.1, -5.6, 1, 2.5, 37.721273, -122.473387),
 (7, 1654752525, 36, 91.3, 127, 57.6, -6, 1, 2.6, 37.721261, -122.473481),
 (8, 1654752525, 44, 113.2, 127, 56.4, -6.2, 1, 2.7, 37.721206, -122.473719),
 (9, 1654752525, 53, 135.8, 128, 54.8, -5.9, 1, 2.6, 37.721182, -122.473969),
 (10, 1654752525, 61, 156, 126, 53.6, -5.9, 1, 2.5, 37.72115, -122.474193)]